<a href="https://colab.research.google.com/github/kattens/PubChem-Data-Handler/blob/main/Blast_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Goal:
we wanna run the blast from our previously created csv file with the targets uniprot ids agains our target (plasmodiom malaria)

In [1]:
#install Bio
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.2 MB/s eta 0:00:00


In [2]:
import csv
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO


In [3]:
path = '/content/drive/MyDrive/target_results.csv'
df = pd.read_csv(path)

In [4]:
df.head()

,PubChem ID,Target Names,Accession IDs,Target Gene Name
0,5330175,"['Tyrosineprotein', 'NTMT1', 'FH', 'Chain', 'N...","['A0A0K2VLS4', 'S4R3J7', 'P07954', 'P0C023', '...","['NTMT1', 'FH', 'NSD2', 'KDR', 'GPX4', 'COMT',..."
1,5311340,"['ID4', 'ALDH1A1', 'EZH2', 'MYC', 'GLA', 'APOB...","['P47928', 'Q5SYQ8', 'Q921E6', 'A0A8A5GQJ2', '...","['ID4', 'ALDH1A1', 'EZH2', 'MYC', 'GLA', 'APOB..."
2,11511120,"['AcylCoA', 'Epidermal', 'Mitogenactivated', '...","['B2BXS0', 'Q9Z0P7', 'L8GZV5', 'P05067', 'Q9ZN...","['NADH', 'MAP', 'CYP2C9', 'NSD2', 'ERBB4', 'GP..."
3,221354,"['CYP2D6', 'lethal', 'ALDH1A1', 'RGS12', 'ALOX...","['P10635', 'A1Z198', 'Q5SYQ8', 'E9Q652', 'I3L1...","['CYP2D6', 'ALDH1A1', 'RGS12', 'ALOX15B', 'HPG..."
4,6806409,[],[],[]


In [ ]:
#loop through the column Accession IDs and run a blast against plasmodiom malaria


In [12]:
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def fetch_uniprot_sequence(accession_id):
    """
    Fetch a sequence from UniProt using the accession ID.
    """
    url = f"https://rest.uniprot.org/uniprotkb/{accession_id}.fasta"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"UniProt query failed for {accession_id} with status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error querying UniProt for {accession_id}: {e}")
        return None

def blast_sequence_from_uniprot(accession_id, identity_threshold=80.0, e_value_threshold=1e-5):
    """
    Fetch a sequence from UniProt, run BLAST, and filter results based on similarity.
    """
    # Fetch sequence from UniProt
    print(f"Fetching sequence for Accession ID: {accession_id} from UniProt...")
    sequence_data = fetch_uniprot_sequence(accession_id)

    if not sequence_data:
        print(f"Sequence for Accession ID {accession_id} could not be found in UniProt.")
        return

    # Run BLAST
    print("Running BLAST...")
    result_handle = NCBIWWW.qblast("blastp", "nr", sequence_data)
    blast_record = NCBIXML.read(result_handle)

    # Check and filter results
    if not blast_record.alignments:
        print(f"No BLAST results found for Accession ID: {accession_id}")
        return

    print(f"Results for Accession ID: {accession_id}")
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            percent_identity = (hsp.identities / hsp.align_length) * 100
            if percent_identity >= identity_threshold and hsp.expect <= e_value_threshold:
                print(f"  Hit ID: {alignment.hit_id}")
                print(f"  Description: {alignment.hit_def}")
                print(f"  Percent Identity: {percent_identity:.2f}%")
                print(f"  E-value: {hsp.expect}")
                print(f"  Score: {hsp.score}")
                print(f"  Query Alignment: {hsp.query[:50]}...")
                print(f"  Subject Alignment: {hsp.sbjct[:50]}...")
                print("-" * 80)



In [14]:
# Example usage:
#Q9Z0P7
#L8GZV5
# Replace "ACCESSION_ID_HERE" with a valid accession ID
# Example usage:
# Replace "ACCESSION_ID_HERE" with a valid accession ID
# Example usage
blast_sequence_from_uniprot("P10635")


Fetching sequence for Accession ID: P10635 from UniProt...
Running BLAST...
Results for Accession ID: P10635
  Hit ID: ref|NP_000097.3|
  Description: cytochrome P450 2D6 isoform 1 [Homo sapiens] >sp|P10635.2| RecName: Full=Cytochrome P450 2D6; AltName: Full=CYPIID6; AltName: Full=Cholesterol 25-hydroxylase; AltName: Full=Cytochrome P450-DB1; AltName: Full=Debrisoquine 4-hydroxylase [Homo sapiens] >gb|AAA53500.1| cytochrome P450 IID6 [Homo sapiens] >gb|AAH75023.1| Cytochrome P450, family 2, subfamily D, polypeptide 6 [Homo sapiens] >gb|AAH75024.1| Cytochrome P450, family 2, subfamily D, polypeptide 6 [Homo sapiens] >gb|AAS55001.1| cytochrome P4502D6 [Homo sapiens] >gb|ABB77895.1| cytochrome P450 2D6 [Homo sapiens]
  Percent Identity: 100.00%
  E-value: 0.0
  Score: 2617.0
  Query Alignment: MGLEALVPLAVIVAIFLLLVDLMHRRQRWAARYPPGPLPLPGLGNLLHVD...
  Subject Alignment: MGLEALVPLAVIVAIFLLLVDLMHRRQRWAARYPPGPLPLPGLGNLLHVD...
---------------------------------------------------------------------

In [13]:
# Example usage
blast_sequence_from_uniprot("L8GZV5")


Fetching sequence for Accession ID: L8GZV5 from UniProt...
Running BLAST...
Results for Accession ID: L8GZV5
  Hit ID: ref|XP_004340546.1|
  Description: mitogenactivated (MAP) kinase [Acanthamoeba castellanii str. Neff] >gb|ELR18507.1| mitogenactivated (MAP) kinase [Acanthamoeba castellanii str. Neff]
  Percent Identity: 100.00%
  E-value: 0.0
  Score: 2416.0
  Query Alignment: MHAPPGPTASMSVTSSSSSASSSSSSISSPSPLLRLASHNLSPRPTTPGH...
  Subject Alignment: MHAPPGPTASMSVTSSSSSASSSSSSISSPSPLLRLASHNLSPRPTTPGH...
--------------------------------------------------------------------------------
